In this Kernel I try for the first time to build a neural network (without the direct specifications of (coursera ;). I use Tensorflow, so that the structure of the Neural Network isbetter visible. At the moment the network works, however it improves only for one step. Than it seems stuck. Any ideas?

## 1. Import necessary packages

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import tensorflow as tf
from tensorflow.python.framework import ops

import skimage.io
from skimage.transform import resize

import matplotlib.pyplot as plt



## 2. Prepare Data

In [ ]:
data = pd.read_csv("../input/train.csv")
data.head()

In [ ]:
#Train/Test Split (https://www.kaggle.com/kmader/transfer-learning-for-human-protein-submission)
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, 
                 test_size = 0.3,         
                 stratify = data['Target'].map(lambda x: x[:3] if '27' not in x else '0'))
train = train.reset_index(drop=True)
test = test.reset_index(drop = True)
test.shape



In [ ]:
#Creating Label-Vector (https://www.kaggle.com/allunia/protein-atlas-exporation-and-baseline)

label_names = {
    0:  "Nucleoplasm",  
    1:  "Nuclear membrane",   
    2:  "Nucleoli",   
    3:  "Nucleoli fibrillar center",   
    4:  "Nuclear speckles",
    5:  "Nuclear bodies",   
    6:  "Endoplasmic reticulum",   
    7:  "Golgi apparatus",   
    8:  "Peroxisomes",   
    9:  "Endosomes",   
    10:  "Lysosomes",   
    11:  "Intermediate filaments",   
    12:  "Actin filaments",   
    13:  "Focal adhesion sites",   
    14:  "Microtubules",   
    15:  "Microtubule ends",   
    16:  "Cytokinetic bridge",   
    17:  "Mitotic spindle",   
    18:  "Microtubule organizing center",   
    19:  "Centrosome",   
    20:  "Lipid droplets",   
    21:  "Plasma membrane",   
    22:  "Cell junctions",   
    23:  "Mitochondria",   
    24:  "Aggresome",   
    25:  "Cytosol",   
    26:  "Cytoplasmic bodies",   
    27:  "Rods & rings"
}


def fill_targets(row):
    row.Target = np.array(row.Target.split(" ")).astype(np.int)
    for num in row.Target:
        name = label_names[int(num)]
        row.loc[name] = 1
    return row

for key in label_names.keys():
    train[label_names[key]] = 0
    test[label_names[key]] = 0
    
train = train.apply(fill_targets, axis=1)
test = test.apply(fill_targets, axis = 1)
test.head()

In [ ]:
X_train = train.iloc[:,0]
Y_train = train.iloc[:, 2:30]
X_test = test.iloc[:,0]
Y_test = test.iloc[:, 2:30]

In [ ]:
Y_test.shape

## 3. Create Data Generator

In [ ]:
#credits: https://www.kaggle.com/byrachonok/pretrained-inceptionresnetv2-base-classifier
def data_generator (dataset, start, end):
    picture = np.empty([end-start, 256, 256, 3])
    for i in range(start, end):
        image_red_ch = skimage.io.imread("../input/train/"+ dataset.iloc[i, 0]+"_red.png")
        image_yellow_ch = skimage.io.imread("../input/train/"+ dataset.iloc[i, 0]+"_yellow.png")
        image_green_ch = skimage.io.imread("../input/train/"+ dataset.iloc[i, 0]+"_green.png")
        image_blue_ch = skimage.io.imread("../input/train/"+ dataset.iloc[i, 0]+"_blue.png")

        image_red_ch += (image_yellow_ch/2).astype(np.uint8) 
        image_green_ch += (image_yellow_ch/2).astype(np.uint8)

        image = np.stack((
            image_red_ch, 
            image_green_ch, 
            image_blue_ch), -1)
        image = resize(image, (256, 256), mode='reflect')
        picture[i-start] = image
    return picture
        

In [ ]:
a = data_generator(train, 0, 12)
print(a.shape)
skimage.io.imshow(a[5])

In [ ]:
#Test whether pictures are already normalized
print(np.max(a))
print(np.min(a))


## 4. Model
Following this Model: https://github.com/Kulbear/deep-learning-coursera/blob/master/Convolutional%20Neural%20Networks/Convolution%20model%20-%20Application%20-%20v1.ipynb****

### 4.1 Helper functions

In [ ]:
ops.reset_default_graph()

In [ ]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    X = tf.placeholder(tf.float32, [None, n_H0, n_W0, n_C0])
    Y = tf.placeholder(tf.float32, [None, n_y])    
    return X, Y

In [ ]:
def initialize_parameters():                    
        
    W1 = tf.get_variable("W1", [4, 4, 3, 8], initializer=tf.contrib.layers.xavier_initializer())
    W2 = tf.get_variable("W2", [2, 2, 8, 16], initializer=tf.contrib.layers.xavier_initializer())
    W3 = tf.get_variable("W3", [2, 2, 16, 16], initializer=tf.contrib.layers.xavier_initializer())

    parameters = {"W1": W1,
                  "W2": W2,
                  "W3": W3}
    
    return parameters

In [ ]:
def forward_propagation(X, parameters):
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters["W3"]
    
    #First convolutiuonal Block
    #### CONV2D: stride of 1, padding 'SAME'
    Z1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
    ##### RELU
    A1 = tf.nn.relu(Z1)
    ##### MAXPOOL: window 8x8, stride 8, padding 'SAME'
    P1 = tf.nn.max_pool(A1, ksize = [1, 8, 8, 1], strides = [1, 8, 8, 1], padding='SAME')

    #Second Convolutional Block
    Z2 = tf.nn.conv2d(P1, W2, strides=[1, 1, 1, 1], padding='SAME')
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool(A2, ksize = [1, 2, 2, 1], strides = [1, 4, 4, 1], padding='SAME')
    print (P2.shape)
    #Third Convolutional Block
    Z3 = tf.nn.conv2d(P2, W3, strides=[1, 1, 1, 1], padding='SAME')
    print (Z3.shape)
    A3 = tf.nn.relu(Z2)
    P3 = tf.nn.max_pool(A3, ksize = [1, 2, 2, 1], strides = [1, 4, 4, 1], padding='SAME')
    print(P3.shape)
    
    # FLATTEN
    P = tf.contrib.layers.flatten(P3)
    #print(P.shape)
    # FULLY-CONNECTED without non-linear activation function (not not call softmax).
    # 6 neurons in output layer. Hint: one of the arguments should be "activation_fn=None" 
    Z3 = tf.contrib.layers.fully_connected(P, 28, activation_fn=None)
    Z3 = tf.nn.sigmoid(Z3)
    #print(Z3.shape)
    ### END CODE HERE ###

    return Z3

In [ ]:
tf.reset_default_graph()

with tf.Session() as sess:
    np.random.seed(1)
    X, Y = create_placeholders(64, 64, 3, 6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    init = tf.global_variables_initializer()
    sess.run(init)
    a = sess.run(Z3, {X: np.random.randn(2,64,64,3), Y: np.random.randn(2,6)})
    print("Z3 = " + str(a))

In [ ]:
#credits: https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
def compute_cost(Z3, Y):
    
    tp = tf.reduce_sum(tf.cast(Y*Z3, 'float'), axis=0)
    fp = tf.reduce_sum(tf.cast((1-Y)*Z3, 'float'), axis=0)
    fn = tf.reduce_sum(tf.cast(Y*(1-Z3), 'float'), axis=0)

    p = tp / (tp + fp + tf.constant(10**-7))
    r = tp / (tp + fn + tf.constant(10**-7))

    f1 = 2*p*r / (p+r+tf.constant(10**-7))
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)

    return 1 - tf.reduce_mean(f1)

In [ ]:
epochs_completed = 0
index_in_epoch = 0
num_examples = train.shape[0]

# serve data by batches
def next_batch(batch_size):
    
    global train
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
     # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        train = train.sample(frac=1).reset_index(drop=True)
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return data_generator(train, start, end), train.iloc[start:end,2:30]

### 4.2 Model

In [ ]:
def model(IMAGE_H = 256, IMAGE_W = 256, TRAINING_ITERATIONS = 2500, 
          BATCH_SIZE = 100, LEARNING_RATE = 1e-4, DISPLAY_STEP = 10):  
    
    ops.reset_default_graph()
    sess = tf.Session()
    costs = []
    #Placeholders
    X,Y = create_placeholders(IMAGE_H, IMAGE_W, 3, 28)
    #Parameters
    parameters = initialize_parameters()
    # Forward propagation
    Z3 = forward_propagation(X, parameters)
    #Cost
    f1 = compute_cost(Z3, Y)
    # Backpropagation
    train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(f1)
    
    init = tf.initialize_all_variables()
    sess.run(init)

    for i in range(TRAINING_ITERATIONS):
        #get new batch
        batch_xs, batch_ys = next_batch(BATCH_SIZE)  
        
        # train on batch
        cost, _ = sess.run([f1, train_step], feed_dict={X: batch_xs, Y: batch_ys})
        costs.append(cost)
        
        if i%DISPLAY_STEP == 0 or (i + 1) == TRAINING_ITERATIONS:
            print('cost %.2f for step %d'%(cost , i))
            
    #Vizualizations
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(LEARNING_RATE))
    plt.show()
    
    return (costs)

In [ ]:
costs = model()